In [9]:
import fcalc

import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

Reading data

In [10]:
all_data = pd.read_csv('Credit_card.csv')
labels = pd.read_csv('Credit_card_label.csv')
all_data = pd.merge(all_data, labels, left_on='Ind_ID', right_on='Ind_ID')
all_data.drop(columns = ['Ind_ID', 'Type_Occupation', 'Birthday_count', 'Mobile_phone', 'Work_Phone', 'Phone', 'EMAIL_ID'], inplace=True)
all_data.dropna(inplace = True)

Binarizing data

In [11]:
def max_qcut(series):
    num_cut = 2
    while(True):
        try:
            pd.qcut(series, num_cut)
        except ValueError:
            return num_cut-1
        else:
            num_cut+=1

In [12]:
qcut = 8

X, y = all_data.drop(columns = 'label').copy(), all_data['label'].copy()

qcut_cols = X.select_dtypes(['int64', 'float64']).columns

for column in qcut_cols:
    X[column] = pd.qcut(X[column], min(qcut, max_qcut(X[column])))

In [13]:
X = pd.get_dummies(X)

In [14]:
X.shape

(1518, 42)

Assesing performance

In [15]:
def cross_val_score(X, y, n_splits, scoring):
    skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
    scores = np.zeros((len(scoring), skf.get_n_splits()))

    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]

        bin_cls = fcalc.classifier.BinarizedBinaryClassifier(X_train.values, y_train.values)
        bin_cls.predict(X_test.values)

        # we always classify non prediction as false prediction
        tmp_predict = np.copy(bin_cls.predictions)
        tmp_predict[tmp_predict == -1] = np.abs(y_test.values[tmp_predict == -1] - 1)

        for j, score in enumerate(scoring):
            scores[j, i] = score(y_test.values, tmp_predict)
    
    return scores.mean(1).round(3)


In [16]:
cross_val_score(X, y, 5, [accuracy_score, f1_score])

array([0.862, 0.302])